In [1]:
# %load_ext autoreload
# %autoreload 2

# Imports

In [1]:
import os
import csv
import tqdm
import itertools
import time
import json
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import Audio
from sklearn.model_selection import ParameterGrid
import librosa

In [2]:
from acestep.pipeline_ace_step import ACEStepPipeline

# Initialization

In [3]:
PARENT_DIR='/home/rileywilliams/soundverse/ACE-step-lowvram/'
style_prompts_path = f'{PARENT_DIR}/scripts/rjw/test_data/styles.csv'
lyrics_path = f'{PARENT_DIR}/scripts/rjw/test_data/lyrics.txt'

In [4]:
ASH_AUDIO_PATH = "/mnt/disks/audio-ai-research-speech-data/avi_ash_PROCESSED"
all_ash_paths = glob.glob(os.path.join(ASH_AUDIO_PATH, '*'))
ASH_TEST_AUDIO_PATH = os.path.join(ASH_AUDIO_PATH, "12_Always On The Run__003.wav")
# melody, sr = torchaudio.load(ASH_TEST_AUDIO_PATH)

# Dev

### Gen Prompts, Seed, and Lyrics 

In [36]:
style_prompts = [
    "Calming lofi beats",
    "Upbeat 80s synthwave with gated drums and analog synths",
    "Hardcore metal with breakdowns and double bass",
    "Baroque-inspired orchestral piece with harpsichord and strings",
    "Ambient drone with slowly evolving textures",
    "West Coast G-funk with talkbox and bass groove",
    "Traditional Japanese shamisen and shakuhachi duet",
    "Experimental glitch with unpredictable rhythmic stutters",
    "Cinematic trailer music with rising tension and orchestral hits",
    "Melodic house with warm synths",
]

lyrics = [
"""
[Intro]
Turn it up, turn it on, let the signal flow
One more tweak, watch the levels grow
Sweep the filter, ride the wave
We’re dialin’ in the sound we crave

[Chorus]
Test the tone, bend the air
Find the shape that’s really there
Turn the knob, adjust the phase
Searchin’ for that sweet upgrade
Echoes bounce, parameters sing
We’re tuning up everything

[Verse]
Latency’s low, the buffer’s clean
Crunching numbers in the machine
From white noise to melody
It’s all just code and symmetry
Drop a beat, reshape the pitch
One hot swap and flip the switch
""",
"[Instrumental]"
]

#### Save style and lyrics prompts once written

In [7]:
# pd.DataFrame(data=style_prompts).to_csv(
#     path_or_buf=style_prompts_path,
#     header=False, index=False
# )
# pd.DataFrame(data=lyrics).to_csv(
#     path_or_buf=lyrics_path,
#     header=False, index=False
# )
# with open(style_prompts_path, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     for _style in style_prompts:
#         writer.writerow([_style])


# with open(lyrics_path, 'w') as f:
#     f.write(lyrics)

#### Load style and lyrics prompts once written

In [5]:
style_prompts = (
    pd.read_csv(filepath_or_buffer=style_prompts_path, header=None)
    .values.flatten()
    .tolist()
)
lyrics = (
    pd.read_csv(filepath_or_buffer=lyrics_path, header=None)
    .values.flatten()
    .tolist()
)

In [6]:
style_prompts = pd.read_csv(
    filepath_or_buffer=style_prompts_path,
    header=None
).values.flatten().tolist()
lyrics = pd.read_csv(
    filepath_or_buffer=lyrics_path,
    header=None
).values.flatten().tolist()
pairs = list(itertools.product(style_prompts, lyrics))

In [30]:
list(enumerate(style_prompts))

[(0, 'Calming lofi beats'),
 (1, 'Upbeat 80s synthwave with gated drums and analog synths'),
 (2, 'Hardcore metal with breakdowns and double bass'),
 (3, 'Baroque-inspired orchestral piece with harpsichord and strings'),
 (4, 'Ambient drone with slowly evolving textures'),
 (5, 'West Coast G-funk with talkbox and bass groove'),
 (6, 'Traditional Japanese shamisen and shakuhachi duet'),
 (7, 'Experimental glitch with unpredictable rhythmic stutters'),
 (8, 'Cinematic trailer music with rising tension and orchestral hits'),
 (9, 'Melodic house with warm synths')]

# Inference Code

In [7]:
BF16=False
TORCH_COMPILE=True
CPU_OFFLOAD=True
OVERLAPPED_DECODE=False # maybe try false?

try:
    del model_demo
except:
    pass
model_demo = ACEStepPipeline(
    # checkpoint_dir=checkpoint_path,
    dtype="bfloat16" if BF16 else "float32",
    torch_compile=TORCH_COMPILE,
    cpu_offload=CPU_OFFLOAD,
    overlapped_decode=OVERLAPPED_DECODE
)

In [8]:
save_path='/home/rileywilliams/soundverse/ACE-step-lowvram/outputs'

In [ ]:
MANUAL_SEEDS=296772417648513
# LYRICS='[Instrumental]'
# PROMPT='lofi beats'
# PROMPT='classical, piano, instrumental'
# PROMPT='soft jazz, light piano'
# PROMPT="edm, synth, bass, euphoric, energetic"
# PROMPT=""
# PROMPT="hardcore metal riffs"
# AUDIO_DURATION = 43
AUDIO_DURATION=-1

for _prompt, _lyrics in pairs:
    _prompt = style_prompts[4]
    _lyrics = lyrics[0]
    output_paths, input_params_json = model_demo(
            format="wav",
            audio_duration=AUDIO_DURATION, #: float = 60.0,
            prompt=_prompt,
            lyrics=_lyrics, #     lyrics: str = None,
            infer_step = 80,
            scheduler_type = "heun",
            # scheduler_type = "pingpong",
            # scheduler_type = "euler", # "euler" is recommended, heun will take more time
            cfg_type = "apg",
            omega_scale = 80.0, # (this is "granularity" in the GUI, defaults to 10)
            manual_seeds=MANUAL_SEEDS, # None,
            guidance_scale = 20.0, # When guidance_scale_lyric > 1 and guidance_scale_text > 1, the guidance scale will not be applied.
            guidance_interval=0.5,
            guidance_interval_decay = 0.0,
            min_guidance_scale = 3.0,
            use_erg_tag=True,
            use_erg_lyric=True,
            use_erg_diffusion=True,
        
            # use_erg_tag: bool = True,
            # use_erg_lyric: bool = True,
            # use_erg_diffusion: bool = True,
            # guidance_scale_text: float = 0.0,
            # guidance_scale_lyric: float = 0.0,
            # guidance_scale_text = 7,
            # guidance_scale_lyric = 2,
        
            # audio2audio_enable = True,
            # ref_audio_strength = 0.10, # 0.18 was good on Gradio site...
            # ref_audio_input = ASH_TEST_AUDIO_PATH,
        save_path=save_path
    )
    print(output_paths)
    break

2025-06-26 21:06:52.867 | INFO     | acestep.pipeline_ace_step:__call__:1488 - Model loaded in 0.00 seconds.
2025-06-26 21:06:54.715 | INFO     | acestep.pipeline_ace_step:__call__:1536 - random audio duration: 106.97140604111421
2025-06-26 21:06:58.267 | INFO     | acestep.pipeline_ace_step:text2music_diffusion_process:847 - cfg_type: apg, guidance_scale: 20.0, omega_scale: 80.0
2025-06-26 21:06:58.270 | INFO     | acestep.pipeline_ace_step:text2music_diffusion_process:1072 - start_idx: 20, end_idx: 60, num_inference_steps: 80
159it [04:58,  1.88s/it]                                                                                                     


In [ ]:
# input_params_json
# output_paths
# input_params_json

In [ ]:
Audio(output_paths)
# Audio(fp)

In [12]:
# fp = os.path.join(
#     '/home/rileywilliams/soundverse/ACE-step-lowvram/'    
#     'outputs/output_20250625235729_0.wav'
# )
# Audio(fp)